CACHEUX Nolan 

In [49]:
# load data from tsp file
import numpy as np

tsp_data = np.loadtxt('../TSPDataset/gr17.2085.tsp')#fri26_d.937 
print(tsp_data.shape)


(17, 17)


In [50]:
import math
import queue
from collections import Counter

def initial_solution(length):
    s = np.random.choice(range(length), length, replace=False) #cities in [0,length]
    return s
    
def objectivefunction(s):
    cost = 0
    for i in range(s.shape[0]-1):
        cost = cost + tsp_data[s[i]][s[i+1]]
    cost = cost + tsp_data[s[-1]][s[0]]
    return cost

def initial_population(size, length):
    population = [] # set initially to empty
    
    # Generate 'size' individuals using random permutations (diversification as per course)
    for _ in range(size):  # loop to generate each individual
        individual = initial_solution(length)  # create a random TSP tour
        population.append(individual)  # add individual to population
    return population

def evaluate_population(pop):
    cost_pop = []
    
    # Evaluate each individual by computing its tour cost (fitness evaluation from the course)
    for individual in pop:
        cost = objectivefunction(individual)
        cost_pop.append(cost)
    return cost_pop   

def tournament_selection (pop, cost_pop, k): # k is the size of selection
    
    # Randomly select k individuals and return the one with the minimal cost
    selected_indices = np.random.choice(len(pop), size=k, replace=False)  # select k indices
    best_index = selected_indices[0]  # initialize best candidate
    for idx in selected_indices:  # compare each candidate's cost
        if cost_pop[idx] < cost_pop[best_index]:
            best_index = idx
    return pop[best_index]  # best indiv among k ones picked randomly from pop

def roulettewheel_selection (pop, cost_pop): 
    
    pop_prob = cost_pop/sum(cost_pop) #divide each cost by the sum of the costs to get a prob
    # Use provided probability distribution (even if minimization ideally would use inverse cost)
    chosen_index = np.random.choice(len(pop), size=1, replace=False, p=pop_prob)  # select one index based on pop_prob
    return pop[chosen_index[0]]  # indiv
    

def replacement_crossover(parent,offspring,cross_pos):
    
    # deal with left side
    start = cross_pos[0]  
    end = cross_pos[1]  
    mapping = {}  # initialize an empty dictionary to hold the gene mapping between offspring and parent in the crossover segment
    
    # Create a mapping for each gene in the crossover segment from the offspring to the corresponding gene in the parent
    for i in range(start, end+1):  
        mapping[offspring[i]] = parent[i]  
    # Use Counter to count occurrences of genes in the crossover segment
    segment_counter = Counter(offspring[start:end+1])  # counts the genes in the crossover segment
    
    # For the left side of the offspring (indices before the crossover segment), correct any duplicates using the mapping
    for i in range(0, start):  
        candidate = offspring[i]  
        while segment_counter[candidate] > 0:  # using Counter to check if candidate exists in the crossover segment
            candidate = mapping[candidate]  # update candidate using the mapping
        offspring[i] = candidate  # assign the corrected candidate back to the offspring at index i
    
    # deal with right side
    # For the right side of the offspring (indices after the crossover segment), correct duplicates similarly
    for i in range(end+1, len(offspring)):  
        candidate = offspring[i]  
        while segment_counter[candidate] > 0:  # use Counter to verify presence in the crossover segment
            candidate = mapping[candidate]  # update candidate using the mapping
        offspring[i] = candidate  # update the offspring with the corrected candidate gene
        
    return offspring  # cleaned offspring

def pmx_crossover(parent_1, parent_2):
    # pick 2 random crossver positions
    cross_pos = np.random.choice(range(1,len(parent_1)-1), 2, replace=False)
    cross_pos = np.sort(cross_pos)
    
    #print('cp {}'.format(cross_pos))
    
    offspring_1 = np.zeros(len(parent_1), dtype=np.uint8)
    offspring_1 [:cross_pos[0]] = parent_1 [0:cross_pos[0]]
    offspring_1 [cross_pos[1]:] = parent_1 [cross_pos[1]:]
    offspring_1 [cross_pos[0]:cross_pos[1]+1] = parent_2 [cross_pos[0]:cross_pos[1]+1]
    final_offspring_1 = replacement_crossover(parent_1,offspring_1,cross_pos)
    
    offspring_2 = np.zeros(len(parent_1), dtype=np.uint8)
    offspring_2 [:cross_pos[0]] = parent_2 [0:cross_pos[0]]
    offspring_2 [cross_pos[1]:] = parent_2 [cross_pos[1]:]
    offspring_2 [cross_pos[0]:cross_pos[1]+1] = parent_1 [cross_pos[0]:cross_pos[1]+1]
    final_offspring_2 = replacement_crossover(parent_2,offspring_2,cross_pos)
    
    #print('o1 {}'.format(final_offspring_1))
    #print('o2 {}'.format(final_offspring_2))
    
    return offspring_1, offspring_2

def mutation(individual):
    
    # Swap two randomly chosen cities in the individual (mutation operator from the course)
    muted = individual.copy()  # copy individual to preserve original
    idx = np.random.choice(len(individual), size=2, replace=False)  # select two distinct indices
    temp = muted[idx[0]]  # swap the two genes
    muted[idx[0]] = muted[idx[1]]
    muted[idx[1]] = temp
    return muted  # muted individual
        
def mutation_decision(prob): 
    flags = ['yes','no']
    decision = np.random.choice(flags, 1, p=[prob, 1-prob]) # select yes or no according to p
    if decision == 'yes':
        return True
    else:
        return False
    
def getbestparentandofs(parent_1,parent_2, ofs_1, ofs_2):
    
    # Compare the costs of parents and offspring and return the best from each pair
    cost_parent1 = objectivefunction(parent_1)
    cost_parent2 = objectivefunction(parent_2)
    cost_ofs1 = objectivefunction(ofs_1)
    cost_ofs2 = objectivefunction(ofs_2)
    best_parent = parent_1 if cost_parent1 < cost_parent2 else parent_2
    best_ofs = ofs_1 if cost_ofs1 < cost_ofs2 else ofs_2
    return best_parent, best_ofs  # best parent among the two and best ofs among the two


In [51]:
def ga(): # genetic algoithm function
    
    pop_size = 200 # must be a power of 2 for code optimization
    individual_size = 17 # if your tsp is 17 cities
    max_generations = 100
    mutation_prob = 0.3
    tournament_size = 8
    elitism_size = 40 # must be a power of 2 for code optimization 
    steady = False

    pop = initial_population(pop_size, individual_size)
    cost_pop = evaluate_population(pop)

    generations = 0
    while (generations < max_generations):
    
        intermediate_pop = []
    
        # prepare elitism
        indices = np.argpartition(cost_pop, elitism_size)[:elitism_size]
        intermediate_pop = [pop[indices[index]] for index in range(elitism_size)] # keep n best individuals
    
        ################ reproduction ################
        for _ in range((pop_size - elitism_size) // 2):
            
            # select tow parents
            parent_1 = tournament_selection(pop, cost_pop, tournament_size)
            parent_2 = tournament_selection(pop, cost_pop, tournament_size)
        
            # crossover
            ofs_1, ofs_2 = pmx_crossover(parent_1, parent_2)
    
            # mutation with probability
            if mutation_decision(mutation_prob):
                ofs_1 = mutation(ofs_1)
            if mutation_decision(mutation_prob):
                ofs_2 = mutation(ofs_2)
                
            # steady state replacement  
            if steady:
                ind_1, ind_2 = getbestparentandofs(parent_1, parent_2, ofs_1, ofs_2)
            else:
                ind_1, ind_2 = ofs_1, ofs_2
        
            intermediate_pop.append(ind_1)
            intermediate_pop.append(ind_2)
    
        # replacement of the population
        pop = np.copy(intermediate_pop)
        np.random.shuffle(pop)
    
        # evaluation 
        cost_pop = evaluate_population(pop)
    
        generations += 1
    
    
    index_best = np.array(cost_pop).argmin()
    return cost_pop[index_best], pop[index_best]


In [52]:
print(ga())

(np.float64(2085.0), array([ 6, 12,  3,  0, 15, 11,  8,  4,  1,  9, 10,  2, 14, 13, 16,  5,  7]))
